<a href="https://colab.research.google.com/github/Niketbhoye/Diabetic-Retinopathy-Detection/blob/main/Diabetic_Retinopathy_Backend_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Attach Drive to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model = load_model('/content/drive/MyDrive/Copy_of_retina_weights.h5')

NameError: name 'load_model' is not defined

# **For Single Model**

In [ ]:

!pip install flask flask-cors pyngrok

import numpy as np
from flask import Flask, request, jsonify
from flask_cors import CORS
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import io
from pyngrok import ngrok

# ngrok authtoken
ngrok.set_auth_token('2nNH8hsrTuUrsY59lNOx4IIqCuN_7fNAZgdFT85EnVRuiHj7N')  #actual token

# Load the trained model (copy_of_retina_weights.h)
model = load_model('/content/drive/MyDrive/Copy_of_retina_weights.h5')

# Preprocess the image
def preprocess_image(image_bytes):
    image = Image.open(io.BytesIO(image_bytes))
    image = image.resize((256, 256))
    image = np.array(image) / 255.0  # Normalize image
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

app = Flask(__name__)
CORS(app)  # Enable CORS to allow requests from React frontend
@app.route('/predict', methods=['POST'])
def predict():
    try:
        if 'image' not in request.files:
            return jsonify({'error': 'No image provided'}), 400

        image_file = request.files['image']
        image_bytes = image_file.read()
        input_image = preprocess_image(image_bytes)

        # Make prediction
        prediction = model.predict(input_image)
        predicted_class = np.argmax(prediction, axis=1)[0]

        # Map predicted class to the actual category name
        class_names = ["Mild", "Moderate", "NO_DR", "Proliferate_DR", "Severe"]
        category = class_names[predicted_class]

        return jsonify({'prediction': category})

    except Exception as e:
        return jsonify({'error': str(e)}), 500  # Returns the error message


if __name__ == '__main__':
    public_url = ngrok.connect(5000)  # Expose the Flask app on port 5000
    print(f"Flask app is running publicly on: {public_url}")
    app.run(host='0.0.0.0', port=5000)


Flask app is running publicly on: NgrokTunnel: "https://230d-34-13-140-49.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Dec/2024 09:17:29] "POST / HTTP/1.1" 404 -


# **For Multi Model**

In [ ]:
!pip install flask flask-cors pyngrok

import numpy as np
from flask import Flask, request, jsonify
from flask_cors import CORS
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import io
from pyngrok import ngrok

# ngrok authtoken
ngrok.set_auth_token('2pbss4U9a0uhMahS4lGoBeEt7ZU_35Zcip2wbNCKwbQpYKwLv')  # actual token

# Loading the four trained models
model_1 = load_model('/content/drive/MyDrive/Copy_of_retina_weights.h5')
model_2 = load_model('/content/drive/MyDrive/best_resnet50_model.h5')
model_3 = load_model('/content/drive/MyDrive/best_model_vgg16.h5')
model_4 = load_model('/content/drive/MyDrive/best_model_inceptionv3.h5')

# Preprocessing the image based on the required size for each model
def preprocess_image(image_bytes, size):
    image = Image.open(io.BytesIO(image_bytes))
    image = image.resize(size)  # Resize to the specified size
    image = np.array(image) / 255.0  # Normalize image
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

app = Flask(__name__)
CORS(app)  # Enable CORS to allow requests from React frontend

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if 'image' not in request.files:
            return jsonify({'error': 'No image provided'}), 400

        image_file = request.files['image']
        image_bytes = image_file.read()

        # Preprocess the image for each model
        input_image_model_1 = preprocess_image(image_bytes, (256, 256))  # Model 1 expects 256x256
        input_image_model_2 = preprocess_image(image_bytes, (224, 224))  # Model 2 expects 224x224
        input_image_model_3 = preprocess_image(image_bytes, (256, 256))  # Model 3 expects 224x224
        input_image_model_4 = preprocess_image(image_bytes, (256, 256))  # Model 4 expects 299x299

        # Make predictions using all four models
        prediction_1 = model_1.predict(input_image_model_1)
        prediction_2 = model_2.predict(input_image_model_2)
        prediction_3 = model_3.predict(input_image_model_3)
        prediction_4 = model_4.predict(input_image_model_4)

        predicted_class_1 = np.argmax(prediction_1, axis=1)[0]
        predicted_class_2 = np.argmax(prediction_2, axis=1)[0]
        predicted_class_3 = np.argmax(prediction_3, axis=1)[0]
        predicted_class_4 = np.argmax(prediction_4, axis=1)[0]

        # Map predicted class to the actual category name
        class_names = ["Mild", "Moderate", "NO_DR", "Proliferate_DR", "Severe"]
        category_1 = class_names[predicted_class_1]
        category_2 = class_names[predicted_class_2]
        category_3 = class_names[predicted_class_3]
        category_4 = class_names[predicted_class_4]

        # Return predictions from all four models
        return jsonify({
            'prediction_model_1': category_1,
            'prediction_model_2': category_2,
            'prediction_model_3': category_3,
            'prediction_model_4': category_4
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500  # Returns the error message


if __name__ == '__main__':
    public_url = ngrok.connect(5000)  # Expose the Flask app on port 5000
    print(f"Flask app is running publicly on: {public_url}")
    app.run(host='0.0.0.0', port=5000)


Flask app is running publicly on: NgrokTunnel: "https://cf4f-34-133-234-111.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 880ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 03:40:52] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 830ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step


INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 04:59:22] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step


INFO:werkzeug:127.0.0.1 - - [02/Dec/2024 05:02:17] "POST /predict HTTP/1.1" 200 -
